In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz

/opt/homebrew/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#removing space after authors 
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [3]:
#This is path to output csv files of the index of each book with they italicized names filtered out
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

In [4]:
def get_set(x):
    if not isinstance(x, list): #is this the best way
        x = []
    return set(x)

#to use after exploding the authors
connectors = {'ex', '&'}
def remove_prantheses(x):
    if isinstance(x, str):
        return re.sub(r'([()])', '', x)
    return x

In [5]:
def process_wcvp(wcvp_df):
    wcvp_df['infraspecies'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['species'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['authors_set'] = wcvp_df['authors'].str.split(' ').apply(lambda x : get_set(x) - connectors)
    wcvp_df_explode = wcvp_df.explode('authors_set')
    wcvp_df_explode['authors_set'] = wcvp_df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return wcvp_df_explode